# Link Prediction

## Preparation

GPU acceleration dengan CUDA

In [1]:
%env NX_CUGRAPH_AUTOCONFIG=True

env: NX_CUGRAPH_AUTOCONFIG=True


In [2]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
!pip install igraph networkit pandas matplotlib seaborn networkx numpy scikit-learn tqdm ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 57.5 MB/s eta 0:00:00


In [3]:
# %pip uninstall torch pykeen
# %pip install torch --index-url https://download.pytorch.org/whl/cu126
!pip install pykeen

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.4/718.4 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import networkx as nx
import pickle
import random
import igraph as ig
import networkit as nk

from itertools import combinations
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_classif
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

## Dataset Preparation

In [6]:
pickle_file_path = 'dataset/amazon_copurchase_graph.pickle'
with open(pickle_file_path, 'rb') as f:
    G = pickle.load(f)

print(G)

DiGraph with 259102 nodes and 1207337 edges


### Features

#### Node Features

In [7]:

print(f"Total Nodes: {G.number_of_nodes()}")


for node, data in list(G.nodes(data=True))[:5]:
    print(f"Node: {node}, Data: {data}")

print()
sample_node = next(iter(G.nodes(data=True)))[1]
print("Node features:", list(sample_node.keys()))

Total Nodes: 259102
Node: 1, Data: {'title': 'Patterns of Preaching: A Sermon Sampler', 'group': 'Book', 'salesrank': 396585.0, 'review_cnt': 2, 'downloads': 2, 'rating': 5.0, 'in_degree': 0, 'out_degree': 4, 'pagerank_centrality': 6.210153588242165e-07, 'betweenness_centrality': 0.0, 'harmonic_closeness_centrality': 0.1442557706580312, 'degree_centrality': 1.5437995221940477e-05, 'community': 10}
Node: 2, Data: {'title': 'Candlemas: Feast of Flames', 'group': 'Book', 'salesrank': 168596.0, 'review_cnt': 12, 'downloads': 12, 'rating': 4.5, 'in_degree': 1, 'out_degree': 4, 'pagerank_centrality': 7.560926314778459e-07, 'betweenness_centrality': 31563.672353370643, 'harmonic_closeness_centrality': 0.1444868764333364, 'degree_centrality': 1.92974940274256e-05, 'community': 10}
Node: 4, Data: {'title': 'Life Application Bible Commentary: 1 and 2 Timothy and Titus', 'group': 'Book', 'salesrank': 631289.0, 'review_cnt': 1, 'downloads': 1, 'rating': 4.0, 'in_degree': 24, 'out_degree': 5, 'page

### Split Dataset

In [9]:
nkG = nk.nxadapter.nx2nk(G)

edges = list(G.edges())
existing_edges = set(edges)

# Sampling dengan Networkit Graph (lebih cepat)
def sample_non_edges_nk(nkG, num_samples):
    non_edges = set()
    nodes = list(G.nodes())

    while len(non_edges) < num_samples:
        u, v = random.sample(nodes, 2)
        if not nkG.hasEdge(u, v):
            non_edges.add((u, v))

    return list(non_edges)

num_samples = len(edges)
non_edges = sample_non_edges_nk(nkG, num_samples)

train_edges, test_edges = train_test_split(edges, test_size=0.2, random_state=42)
train_non_edges = random.sample(non_edges, len(train_edges))
test_non_edges = random.sample(non_edges, len(test_edges))

G_train = nx.Graph()
G_train.add_nodes_from(G.nodes())
G_train.add_edges_from(train_edges)

print(f"Train Edges: {len(train_edges)}, Test Edges: {len(test_edges)}")
print(f"Train Non-Edges: {len(train_non_edges)}, Test Non-Edges: {len(test_non_edges)}")

Train Edges: 965869, Test Edges: 241468
Train Non-Edges: 965869, Test Non-Edges: 241468


## Heuristic Link Prediction

In [25]:
def heuristic_score(G, node_pairs, method):
    scores = []
    for u, v in tqdm(node_pairs, desc=f"Computing {method} scores", disable=True):
        if method == "common_neighbors":
            score = len(list(nx.common_neighbors(G, u, v)))
        elif method == "jaccard":
            score = list(nx.jaccard_coefficient(G, [(u, v)]))[0][2]
        elif method == "adamic_adar":
            score = list(nx.adamic_adar_index(G, [(u, v)]))[0][2]
        elif method == "preferential_attachment":
            score = list(nx.preferential_attachment(G, [(u, v)]))[0][2]
        else:
            raise ValueError("Method not recognized")
        scores.append(score)
    return scores


In [24]:
# Metrik evaluasi ranking problem
def precision_at_k(y_true, y_scores, k):
    sorted_indices = np.argsort(y_scores)[::-1]
    top_k = sorted_indices[:k]
    return np.mean(y_true[top_k])

def recall_at_k(y_true, y_scores, k):
    sorted_indices = np.argsort(y_scores)[::-1]
    top_k = sorted_indices[:k]
    return np.sum(y_true[top_k]) / np.sum(y_true)

def mean_average_precision(y_true, y_scores):
    sorted_indices = np.argsort(y_scores)[::-1]
    relevant = np.cumsum(y_true[sorted_indices])
    precision_at_i = relevant / (np.arange(len(y_true)) + 1)
    return np.sum(precision_at_i * y_true[sorted_indices]) / np.sum(y_true)

def f1_beta_at_k(y_true, y_scores, k, beta=1):
    precision_k = precision_at_k(y_true, y_scores, k)
    recall_k = recall_at_k(y_true, y_scores, k)

    if precision_k + recall_k == 0:
        return 0.0

    beta_sq = beta ** 2
    return (1 + beta_sq) * (precision_k * recall_k) / ((beta_sq * precision_k) + recall_k)



## Graph Convolutional Network (GCN) Link Prediction

In [13]:
!pip install torch-geometric


Model yang dibangun adalah sebagai berikut.

1. **Lapisan Input**
    - **Learnable Node Embeddings**: `torch.nn.Embedding(num_nodes, 32)`
        - Setiap node memiliki embedding berdimensi 32.

2. **Lapisan Konvolusi Graf** (GCN)
    - **GCNConv(32 → 64)** → Konvolusi Awal
        - Aktivasi ReLU
        - Dropout (p=0.5)

    - **GCNConv(64 → 64)** → Konvolusi Tambahan
        - Aktivasi ReLU
        - Dropout (p=0.5)

    - **GCNConv(64 → 32)** → Konvolusi Akhir

3. **Edge Decoder** (Link Prediction)

    - **Dot Product Decoder**

        - Menghitung $$score(u,v)=z_u * z_v$$
        - Menghasilkan skor probabilitas untuk setiap hubungan (edge). (logit score)

4. **Loss Function**

  - **Binary Cross Entropy dengan Logits** (`BCEWithLogitsLoss`).
  - **Regularisasi L2** (`torch.norm(z, p=2)`).

5. **Optimisasi**
    - **Optimizer**: AdamW
      - Learning rate: 0.002
      - Weight decay: 5e-4
    - **Learning Rate Scheduler**: `StepLR(step_size=15, gamma=0.7)`
      - Mengurangi learning rate setiap 15 epoch.

In [20]:
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score

node_to_idx = {node: idx for idx, node in enumerate(G.nodes())}
edge_index = torch.tensor(
    [[node_to_idx[u], node_to_idx[v]] for u, v in G.edges() if u in node_to_idx and v in node_to_idx],
    dtype=torch.long
).t().contiguous()

# Use learnable node embeddings with higher dimension
num_nodes = G.number_of_nodes()
embedding_dim = 32
x = torch.nn.Embedding(num_nodes, embedding_dim).weight

data = Data(x=x, edge_index=edge_index)

class GNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, out_channels)
        self.dropout = torch.nn.Dropout(p=0.5)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.dropout(x)
        return self.conv3(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)

# Initialize model and optimizer
model = GNN(in_channels=embedding_dim, hidden_channels=64, out_channels=32)  # Increased hidden size
optimizer = torch.optim.AdamW(model.parameters(), lr=0.002, weight_decay=5e-4)  # Adjusted learning rate & weight decay
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.7)  # More frequent LR decay

# Training loop
for epoch in range(1, 101):
    model.train()
    optimizer.zero_grad()

    z = model.encode(data.x, data.edge_index)
    pos_edge_index = data.edge_index
    neg_edge_index = negative_sampling(
        edge_index=pos_edge_index, num_nodes=data.num_nodes, num_neg_samples=2 * len(pos_edge_index[0])  # Increased negatives
    )

    pos_out = model.decode(z, pos_edge_index)
    neg_out = model.decode(z, neg_edge_index)

    out = torch.cat([pos_out, neg_out], dim=0)
    labels = torch.cat([torch.ones(pos_out.size(0)), torch.zeros(neg_out.size(0))], dim=0)

    loss = F.binary_cross_entropy_with_logits(out, labels)
    loss += 0.001 * torch.norm(z, p=2)  # L2 regularization
    loss.backward()
    optimizer.step()
    scheduler.step()

    # if epoch % 10 == 0:
    #     print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')


Epoch: 001, Loss: 2.5200
Epoch: 002, Loss: 2.0196
Epoch: 003, Loss: 1.6920
Epoch: 004, Loss: 1.4797
Epoch: 005, Loss: 1.3457
Epoch: 006, Loss: 1.2640
Epoch: 007, Loss: 1.2084
Epoch: 008, Loss: 1.1721
Epoch: 009, Loss: 1.1470
Epoch: 010, Loss: 1.1260
Epoch: 011, Loss: 1.1085
Epoch: 012, Loss: 1.0930
Epoch: 013, Loss: 1.0789
Epoch: 014, Loss: 1.0645
Epoch: 015, Loss: 1.0503
Epoch: 016, Loss: 1.0370
Epoch: 017, Loss: 1.0272
Epoch: 018, Loss: 1.0173
Epoch: 019, Loss: 1.0085
Epoch: 020, Loss: 0.9989
Epoch: 021, Loss: 0.9905
Epoch: 022, Loss: 0.9807
Epoch: 023, Loss: 0.9732
Epoch: 024, Loss: 0.9645
Epoch: 025, Loss: 0.9583
Epoch: 026, Loss: 0.9498
Epoch: 027, Loss: 0.9428
Epoch: 028, Loss: 0.9372
Epoch: 029, Loss: 0.9303
Epoch: 030, Loss: 0.9234
Epoch: 031, Loss: 0.9182
Epoch: 032, Loss: 0.9136
Epoch: 033, Loss: 0.9100
Epoch: 034, Loss: 0.9051
Epoch: 035, Loss: 0.9024
Epoch: 036, Loss: 0.8981
Epoch: 037, Loss: 0.8948
Epoch: 038, Loss: 0.8917
Epoch: 039, Loss: 0.8879
Epoch: 040, Loss: 0.8842


In [26]:
# Evaluation
model.eval()
with torch.no_grad():
    z = model.encode(data.x, data.edge_index)
    test_edges_tensor = torch.tensor(
        [[node_to_idx[u], node_to_idx[v]] for u, v in test_edges if u in node_to_idx and v in node_to_idx],
        dtype=torch.long
    ).t().contiguous()
    test_non_edges_tensor = torch.tensor(
        [[node_to_idx[u], node_to_idx[v]] for u, v in test_non_edges if u in node_to_idx and v in node_to_idx],
        dtype=torch.long
    ).t().contiguous()

    test_pos_out = model.decode(z, test_edges_tensor)
    test_neg_out = model.decode(z, test_non_edges_tensor)

    out_test = torch.cat([test_pos_out, test_neg_out], dim=0)
    labels_test = torch.cat([torch.ones(test_pos_out.size(0)), torch.zeros(test_neg_out.size(0))], dim=0)

    k = 100000
    probabilities = out_test.cpu().numpy()
    labels_np = labels_test.cpu().numpy()

    roc_auc = roc_auc_score(labels_np, probabilities)
    ap_score = average_precision_score(labels_np, probabilities)
    precision_at_k_val = precision_at_k(labels_np, probabilities, k)
    recall_at_k_val = recall_at_k(labels_np, probabilities, k)
    map_score = mean_average_precision(labels_np, probabilities)
    f1_k_val = f1_beta_at_k(labels_np, probabilities, k)

    print("{:<25} {:>10} {:>15} {:>15} {:>15} {:>15} {:>15}".format("Model", "AUC-ROC", "AP Score", "Prec@100k", "Rec@100k", "MAP", "F1@100k"))
    print("=" * 105)
    print("{:<25} {:>10.6f} {:>15.6f} {:>15.6f} {:>15.6f} {:>15.6f} {:>15.6f}".format(
        "GNN", roc_auc, ap_score, precision_at_k_val, recall_at_k_val, map_score, f1_k_val
    ))


Model                        AUC-ROC        AP Score       Prec@100k        Rec@100k             MAP         F1@100k
GNN                         0.763832        0.782446        0.863880        0.357762        0.782446        0.505980


In [28]:
# Save the model
torch.save(model.state_dict(), "gnn_model.pth")
print("Success!")

Success!
